In [56]:
import requests
from bs4 import BeautifulSoup
from src.utils.config import load_config

In [57]:
config = load_config()

In [ ]:
urls = config['sources']['seed_urls']
allowed_domains = config['sources']['allowed_domains']
legal_websites = []
ill_websites = {}

for url in urls:
    for allowed_domain in allowed_domains:
        if allowed_domain in url:
            legal_websites.append(url)
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            file_name = 'file' +str(url) + '.html'
            try:
                with open(file_name, 'w', encoding = 'utf-8') as file:
                    file.write(soup.prettify())
                    print('successfully saved HTML to', file_name)
                    
            except IOError as e:
                print(f"Error saving file : {e}")

        else:
            ill_websites[allowed_domain] = url
            

TypeError: unsupported operand type(s) for +=: 'NoneType' and 'int'